# Assignment 1

In [3]:
# Set up all necessary imports
!pip install hfst
import hfst
from IPython.display import Image
from hfst import compile_lexc_file
from hfst import regex, HfstTransducer

### Lexicon (explained)
Multichar Symbols - tags that describe the noun forms.

```
Multichar_Symbols: +N +Sg +Pl +Nom +Gen +Dat +Acc +Ins +Pre

Includes the cases in the Russian language and plural and singular forms for words.
Russian has Nominative, Genitive, Dative, Accusative, Instrumental, Prepositional forms for nouns.

```

Root Lexicon
```
LEXICON Root
Noun ;
```


Noun Lexicon (listing words)

```
LEXICON Noun
птиц N1decl;  bird (belong to 1st declanation)
тыкв N1decl;  pumpkin
изб N1decl;  hut
друг N2decl; friend (2nd declanation)
дуб N2decl;  oak tree
отец N2decl; father
волк N2decl;  wolf
мыш N3decl;  mouse (3rd declanation)
ноч N3decl;  night
сирен N3decl;  lilac

```



#### Declanation Rules (How the nouns change)

```
! In the first declanation, these are typically feminine nouns ending in -а or -я.

LEXICON N1decl
+N+Sg+Nom:^а #;
+N+Sg+Gen:^ы  #;
+N+Sg+Dat:^е  #;
+N+Sg+Acc:^у  #;
+N+Sg+Ins:^ой  #;
+N+Sg+Pre:^е  #;
+N+Pl+Nom:^ы  #;
+N+Pl+Gen:^  #;
+N+Pl+Dat:^ам  #;
+N+Pl+Acc:^ #;
+N+Pl+Ins:^ами  #;
+N+Pl+Pre:^ах  #;

! In the second declanation, these are typically masculine nouns ending in a consonant or -й, as well as some neutral nouns.

LEXICON N2decl
+N+Sg+Nom:^ #;
+N+Sg+Gen:^a  #;
+N+Sg+Dat:^у  #;
+N+Sg+Acc:^а #;
+N+Sg+Ins:^ом  #;
+N+Sg+Pre:^е  #;
+N+Pl+Nom:^ы  #;
+N+Pl+Gen:^ов  #;
+N+Pl+Dat:^ам  #;
+N+Pl+Acc:^ов #;
+N+Pl+Ins:^ами  #;
+N+Pl+Pre:^ах  #;

! In the third declanation, these are typically feminine nouns ending in ь.

LEXICON N3decl
+N+Sg+Nom:^ь  #;
+N+Sg+Gen:^и  #;
+N+Sg+Dat:^и  #;
+N+Sg+Acc:^ь  #;
+N+Sg+Ins:^ью  #;
+N+Sg+Pre:^и  #;
+N+Pl+Nom:^и  #;
+N+Pl+Gen:^ей  #;
+N+Pl+Dat:^ям  #;
+N+Pl+Acc:^и  #;
+N+Pl+Ins:^ями  #;
+N+Pl+Pre:^ях  #;
```


## Compiling the lexicon script

In [8]:
# Upload lexicon from repository
!wget https://raw.githubusercontent.com/lotockatest/FST_task/refs/heads/main/Russian.lexc

--2025-04-01 19:14:45--  https://raw.githubusercontent.com/lotockatest/FST_task/refs/heads/main/Russian.lexc
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1035 (1.0K) [text/plain]
Saving to: ‘Russian.lexc.3’

Russian.lexc.3      100%[===================>]   1.01K  --.-KB/s    in 0s      

2025-04-01 19:14:45 (55.3 MB/s) - ‘Russian.lexc.3’ saved [1035/1035]



In [11]:
generator = hfst.compile_lexc_file('Russian.lexc')
# print(generator)
print(generator.lookup('изб+N+Sg+Nom'))

(('изб^а@_EPSILON_SYMBOL_@', 0.0),)


In [10]:
# Insertion
# Insert "е" after consonants (like г, к, х, ж, ч, ш, щ) when these consonants are followed by one of the vowels а, у, or ы and add ^ mark.
Insert = regex("[. .] -> е || [г | к | х | ж | ч | ш | щ] %^ _ а | у | ы")
print(Insert.lookup("волк^и"))

(('волк^и', 0.0),)


In [12]:
# Replacement
# Removes the "е" in "отец" to produce "отц". It is a phonological simplification rule for ease of pronounciation in Russian, which applies to the specific word.
Replace = hfst.regex("[отец -> отц]")
print(Replace.lookup("отец^ом"))
print(Replace.lookup("отец^у"))

(('отц^ом', 0.0),)
(('отц^у', 0.0),)


In [13]:
# Exceptions
# Exception for the word "друг". In its plural form it is "друзья" and does not follow typical second declanation rules in plural.
Exceptions = hfst.regex("д р у г %^ ы -> д р у з ь я")
print(Exceptions.lookup("друг^ы"))

(('друзья', 0.0),)


In [21]:
# Deleting
# Ensure the text _EPSILON_SYMBOL_ is removed.
Cleanup = hfst.regex('"@_EPSILON_SYMBOL_@" -> ""')
print(Cleanup.lookup("ноч^и"))

(('ноч^и', 0.0),)
